# A very simpel model for spreading of infectious diseases
This is an experimental model. Inspirered by https://triplebyte.com/blog/modeling-infectious-diseases. 

The model is purely for testing the capabilities of ModelFlow, the parameters selected are for ilustration of the dynamic and are not actual estimates. 

This is a Jupyter Notebook running Python. 

The notebook is located on github here: https://github.com/IbHansen/Modelflow_test

Feel free to use this notebook. **To run the notebook** find the menu line above and select **cell>run all** 

THE Notebook IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

The model is specified as equations in the Python code below (you don't have to understand the Python code). The equations defines the daily transition between the states which the population can exist in:

 - susciptible
 - exposed
 - infectious
 - recovered
 - dead 

# Make experiments 
The model will run with a baseline scenario and you can inspect the results. 

Then you can use the sliders to change the scenario.
You can make your own scenario. Use the sliders to define an scenario. Then press the **Run scenario** button. The model will solve, and the results can be inspected.

For instance, try to increase *Daily rate of contact* to 6 and press **Run scenario**  and watch how the values changes.

You can change the scenario name. After **Run scenario** a graph for each scenario will be shown. 

Again, you don't have to understand the Python code. 

In [3]:
import pandas as pd

from modelclass import model
model.modelflow_auto()

# Specify the model 
#    Some conventions:
#       - (-1) after a variable means the value the day before.
#       - diff means the change in variable from the day before

rcorona = '''\
             infection_rate        = min(rate_contact * probability_transmision * infectious(-1) / population(-1),1.0) 
             new_exposed           = infection_rate * susceptible
             diff(exposed)         = new_exposed - new_infectious + exo_exposed

             new_infectious        = incubation_rate * exposed 
             diff(infectious)      = new_infectious - new_recovered  - new_dead

             diff(susceptible)     = -new_exposed  

             new_recovered         = recovery_rate * infectious(-1)
             diff(recovered)       = new_recovered 
             
             new_dead              = dead_rate * infectious(-1)
             diff(dead)            = new_dead 
             diff(population)      = -diff(dead)             
'''
mcorona = model.from_eq(rcorona)       # create a model instance which can solve the model 

DAYS = 500                                # number of days the model will run 
basedf = pd.DataFrame(index=range(DAYS))  # make an empty dataframe with DAYS rows
basedf.index.name = 'Day'

# Define user interface 
cow = mcorona.inputwidget(basedf=basedf,modelopt={'silent':0,'max_iterations':1000,'first_test':10},
                  slidedef = {
                     'Population            '     :{'var' : 'POPULATION SUSCEPTIBLE', 'min' : 0.0, 'max' : 10_000_000, 'value' : 1_000_000,'step':100_000,'op':'=start-','dec':0},
                     'Number of infected t=1'     :{'var' : 'EXO_EXPOSED',            'min' : 0.0, 'max' : 1000,       'value' : 300,      'step':10,'op':'=impulse','dec':0},
                     'Daily incubation rate'      :{'var' : 'INCUBATION_RATE',        'min' : 0.0, 'max' : 1.0,        'value' : 0.1,                'op':'='},
                     'Daily death rate'           :{'var' : 'DEAD_RATE',              'min' : 0.0, 'max' : 1.0,        'value' : 0.01,               'op':'=' },
                     'Daily recovery rate'        :{'var' : 'RECOVERY_RATE',          'min' : 0.0, 'max' : 1.0,        'value' : 0.1,                'op':'='},
                     'Daily rate of contact'      :{'var' : 'RATE_CONTACT',           'min' : 0.0, 'max' : 30,         'value' : 4,'step':0.1,         'op':'='},
                     'Probability of transmission':{'var' : 'PROBABILITY_TRANSMISION','min' : 0.0, 'max' : 1.0,        'value' : 0.05,'step':0.005,  'op':'=','dec':3},
                             },
                 varpat='infectious recovered dead new_infectious new_recovered new_dead',showvar=False);